In [3]:
import sys
sys.path.append('/workspace/Documents')
import os
import nibabel as nb
import numpy as np
import pandas as pd
import shutil
import torch
import torch.nn.functional as F
from scipy.ndimage import center_of_mass
import Diffusion_motion_field.functions_collection as ff
import Diffusion_motion_field.Data_processing as Data_processing
from Diffusion_motion_field.denoising_diffusion_pytorch.denoising_diffusion_pytorch.conditional_EDM_warp import *

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/Documents/Diffusion_motion_field/denoising_diffusion_pytorch/denoising_diffusion_pytorch/conditional_diffusion_3D.py:882: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


#### Step 1: split the data from 4d to 3d
now the data is in format [x,y,z,t], split into 3Ds

In [5]:
main_path = '/mnt/camca_NAS/4DCT/mgh_data/nii-images/'
folders = ff.sort_timeframe(ff.find_all_target_files(['Pull_4/*'],main_path),0,'_')
for f in folders:
    print(f)
    file = os.path.join(f,'img.nii.gz')
    if os.path.isfile(os.path.join(f,'img-nii','10.nii.gz')):
        print('Already processed')
        continue
    img,affine = nb.load(file).get_fdata(), nb.load(file).affine
    tf_num = img.shape[-1]; print('Number of time frames:',tf_num)
    
    for i in range(0,tf_num):
        ff.make_folder([os.path.join(f,'img-nii')])
        nb.save(nb.Nifti1Image(img[:,:,:,i],affine),os.path.join(f,'img-nii',str(i)+'.nii.gz'))  

/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_144
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_145
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_146
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_147
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_148
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_149
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_150
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_151
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_152
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_153
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_154
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_155
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-images/Pull_4/case_156
Already processed
/mnt/camca_NAS/4DCT/mgh_data/nii-image

#### Step 2: get a patient list to start
we will prune this list by exclusion criteria afterwards

In [2]:
main_path = '/mnt/camca_NAS/4DCT/mgh_data/nii-images/'
folders = ff.sort_timeframe(ff.find_all_target_files(['Pull_*/*'],main_path),0,'_')
correspondance_info = pd.read_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_name_correspondance.xlsx')
result = []
for f in folders:
    patient_id = os.path.basename(f)
    patient_class = os.path.basename(os.path.dirname(f))

    row = correspondance_info.loc[correspondance_info['patient_id'] == patient_id]
    case_original_name = row['case_original_name'].values[0]

    # load the image
    file = os.path.join(f,'img-nii','0.nii.gz')
    img = nb.load(file)
    # pixel dimension
    pix_dim = img.header.get_zooms()[0:3]
    # image size
    shape, tf_num = img.shape, len(ff.find_all_target_files(['img-nii/*'],f))
    # print('patient_class:', patient_class, ' patient_id:', patient_id, ' pix_dim:', pix_dim, ' shape:', shape, ' tf_num:', tf_num)
    result.append([patient_class, patient_id, pix_dim, shape, tf_num, case_original_name])

df = pd.DataFrame(result, columns=['patient_class', 'patient_id', 'pix_dim', 'shape', 'timeframes', 'case_original_name'])
df.to_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_original.xlsx', index=False)

#### Step 3: image resample

In [1]:
# main_path = '/mnt/camca_NAS/4DCT/mgh_data/nii-images/'
# folders = ff.sort_timeframe(ff.find_all_target_files(['Pull_4/*'],main_path),0,'_')
# for f in folders:
#     patient_id = os.path.basename(f)
#     patient_class = os.path.basename(os.path.dirname(f))

#     folder = os.path.join(main_path,patient_class,patient_id,'img-nii')
#     files = ff.sort_timeframe(ff.find_all_target_files(['*.nii.gz'],folder),2)
#     print(patient_class, patient_id)
#     for f in files:
#         save_path = os.path.join(main_path,patient_class,patient_id,'img-nii-resampled-1.5mm')
#         if os.path.isfile(os.path.join(save_path, os.path.basename(f))):
#             continue
#         img = nb.load(f)
#         img_data = img.get_fdata()
#         lr_dim = [1.5, 1.5, 1.5]
#         hr_resample = Data_processing.resample_nifti(img, order=3,  mode = 'nearest',  cval = np.min(img_data), in_plane_resolution_mm=lr_dim[0], slice_thickness_mm=lr_dim[-1])
#         ff.make_folder([save_path])
#         nb.save(hr_resample,os.path.join(save_path,os.path.basename(f)))

#### Step 4: run segmentation u-net

### Step 5: exclude the cases
exclusion criteria here:

(1) no segmentation 

(2) if when crop to [160,160,96] LV is not fully included, we need to shift the data; if after shift still not fully include, exclude

(3) shape consistency in all time frames

(4) time frame order is correct

In [3]:
df = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_original.xlsx'))
result = []
for i in range(0,df.shape[0]):
    patient_class = df.iloc[i]['patient_class']
    patient_id = df.iloc[i]['patient_id']
  
    print('patient_class:', patient_class, ' patient_id:', patient_id)
    # load segmentation
    seg_folder = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg/',patient_class,patient_id)
    seg_files = ff.sort_timeframe(ff.find_all_target_files(['*.nii.gz'],seg_folder),2,'_')
    tf_num = len(seg_files)
    if tf_num == 0:
        continue
    
    # check shape consistency
    seg_volumes = []
    shape_consistency = True
    for ii in range(0, seg_files.shape[0]):
        img = nb.load(seg_files[ii]).get_fdata(); img = np.round(img).astype(np.uint8)
        if ii == 0:
            reference_shape = img.shape
            affine = nb.load(seg_files[ii]).affine
        current_shape = img.shape
        if shape_consistency == True:
            if reference_shape != current_shape:
                shape_consistency = False
        seg_volumes.append(img)
    seg_volumes = np.transpose(np.asarray(seg_volumes),(1,2,3,0))

    # get a list of LV volumes (pixel value = 1) for all timeframes
    LV_volumes = [np.sum(seg_volumes[:,:,:,i]==1) for i in range(0,seg_volumes.shape[-1])]
    ejection_fraction = (LV_volumes[0] - np.min(LV_volumes)) / LV_volumes[0] * 100

    # assert whether there is a <500 in LV_volumes
    have_segmentaiton = [LV_volumes[i] > 500 for i in range(0,tf_num)]
    have_segmentation = False if False in have_segmentaiton else True

    # assert time frame order is correct
    min_index = np.where(np.array(LV_volumes) == np.min(LV_volumes))[0][0]
    max_index = np.where(np.array(LV_volumes[0:10]) == np.max(LV_volumes[0:10]))[0][0]

    # crop and pad
    if have_segmentation == False:
        include_all_LV, LV_pixel_diff, need_to_shift = None,None,None
    else:
        seg_ref = nb.load(os.path.join(seg_folder,'pred_s_0.nii.gz')).get_fdata()
        if len(seg_ref.shape) == 4:
            seg_ref = seg_ref[...,0]
        seg_ref = np.round(seg_ref); seg_ref[seg_ref!=1]= 0

        # compute the center of mass of the LV 
        com = center_of_mass(seg_ref)
        x_com, y_com, z_com = int(com[0]), int(com[1]), int(com[2])

        # shift 
        shift_x = x_com - seg_ref.shape[0]//2   
        shift_y = y_com - seg_ref.shape[1]//2 
        shift_z = z_com - seg_ref.shape[2]//2

        seg_ref_pad = Data_processing.crop_or_pad(seg_ref,[160,160,96],value = 0)
        
        if np.sum(seg_ref) == np.sum(seg_ref_pad):
            include_all_LV = True
            LV_pixel_diff = 0
            need_to_shift = False   
        else:
            print('change!!!!!!!, not include all LV pixels')
            seg_ref_shifted = Data_processing.translate_image(seg_ref, [shift_x, shift_y, shift_z])
            seg_ref_pad = Data_processing.crop_or_pad(seg_ref_shifted,[160,160,96],value = 0)
            if np.sum(seg_ref_shifted) == np.sum(seg_ref_pad):
                print('after shift, include all LV pixels')
                include_all_LV, LV_pixel_diff, need_to_shift = True, 0, True
                # shift the image and segmentation
                img_files = ff.sort_timeframe(ff.find_all_target_files(['img-nii-resampled-1.5mm/*'],os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id)),2)
                for img_file in img_files:
                    img = nb.load(img_file).get_fdata()
                    img_shifted = Data_processing.translate_image(img, [shift_x, shift_y, shift_z])
                    ff.make_folder([os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm-shifted')])
                    nb.save(nb.Nifti1Image(img_shifted, affine), os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images/',patient_class,patient_id,'img-nii-resampled-1.5mm-shifted',os.path.basename(img_file)))
                # shift the segmentation
                seg_files2 = ff.sort_timeframe(ff.find_all_target_files(['*.nii.gz'],seg_folder),2,'_')
                for seg_file in seg_files2:
                    img = nb.load(seg_file).get_fdata()
                    img_shifted = Data_processing.translate_image(img, [shift_x, shift_y, shift_z])
                    ff.make_folder([os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg-shifted',patient_class,patient_id)])
                    nb.save(nb.Nifti1Image(img_shifted, affine), os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg-shifted',patient_class,patient_id,os.path.basename(seg_file)))
            else:
                print('after shift, no change!')
                include_all_LV = False
                LV_pixel_diff = np.sum(seg_ref) - np.sum(seg_ref_pad)
                need_to_shift = False

    result.append([patient_class, patient_id, have_segmentation, LV_volumes , ejection_fraction, max_index, min_index, include_all_LV, LV_pixel_diff, need_to_shift])

df2 = pd.DataFrame(result, columns=['patient_class', 'patient_id', 'have_segmentation', 'LV_volumes','ejection_fraction', 'max_index_in_first_10_tf', 'min_index', 'include_all_LV', 'LV_pixel_diff', 'need_to_shift'])
# merge df and df2 according to patient id
df = pd.merge(df2, df, on=['patient_class', 'patient_id'], how='left')
df.to_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_original_screening.xlsx', index=False)

patient_class: Pull_1  patient_id: case_1
patient_class: Pull_1  patient_id: case_2
patient_class: Pull_1  patient_id: case_3
patient_class: Pull_1  patient_id: case_5
patient_class: Pull_1  patient_id: case_6
patient_class: Pull_1  patient_id: case_7
patient_class: Pull_1  patient_id: case_8


/tmp/ipykernel_9074/3106833436.py:32: RuntimeWarning: invalid value encountered in long_scalars
  ejection_fraction = (LV_volumes[0] - np.min(LV_volumes)) / LV_volumes[0] * 100


patient_class: Pull_1  patient_id: case_9
patient_class: Pull_1  patient_id: case_11
patient_class: Pull_1  patient_id: case_12
patient_class: Pull_1  patient_id: case_13
patient_class: Pull_1  patient_id: case_14
patient_class: Pull_1  patient_id: case_15
patient_class: Pull_1  patient_id: case_17
patient_class: Pull_1  patient_id: case_19
patient_class: Pull_1  patient_id: case_20
patient_class: Pull_1  patient_id: case_21
patient_class: Pull_1  patient_id: case_22
patient_class: Pull_1  patient_id: case_23
patient_class: Pull_1  patient_id: case_24
patient_class: Pull_1  patient_id: case_25
patient_class: Pull_1  patient_id: case_26
patient_class: Pull_1  patient_id: case_27
patient_class: Pull_1  patient_id: case_28
patient_class: Pull_1  patient_id: case_29
patient_class: Pull_1  patient_id: case_30
patient_class: Pull_1  patient_id: case_32
patient_class: Pull_1  patient_id: case_33
patient_class: Pull_1  patient_id: case_34
patient_class: Pull_1  patient_id: case_35
patient_clas

### Step 6

In [5]:
### let's exclude cases
df = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_original_screening.xlsx'))
exclude_index_list = []
for i in range(0, df.shape[0]):
    exclude = False
    row = df.iloc[i]
    if row['have_segmentation'] == False:
        exclude = True
    else:
        if row['include_all_LV'] == False:
            exclude = True
        if row['min_index'] < 5 or row['min_index'] > 13:
            exclude = True
        if row['max_index_in_first_10_tf'] > 5:
            exclude = True
        if row['timeframes'] < 10:
            exclude = True
    if exclude == True:
        exclude_index_list.append(i)

df = df.drop(exclude_index_list)
df['batch'] = 0
df = df.reset_index(drop=True)
df.to_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_selected.xlsx', index=False)

#### Step 7: exclude some time frames / or some cases
based on information we got from step 5/6, there are cases which have certain time frames (usually at the beginning frames and last frames) with very low quality, we should exclude them. 

尤其关注：
max_index_in_first_10tf （可以删掉这些timeframes）/ min_index/ 比较第一帧和最后几帧的LV volume（可以删掉后面几帧或者调整顺序）

now we should manually correect/exclude them. 

after correction, we should run step 2，4，5 again.

In [13]:
# pull = 'Pull_4'
# case_id = 'case_205'
# folder = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images',pull,case_id)
# new_times = np.asarray([2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
# new_folder = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images',pull,case_id,'img-nii-new')
# new_folder_resample = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/nii-images', pull,case_id,'img-nii-resampled-1.5mm-new')
# ff.make_folder([new_folder,new_folder_resample])
# for t in range(0,new_times.shape[0]):
#     original_time = new_times[t]
#     new_time = t
#     shutil.copy2(os.path.join(folder,'img-nii',str(original_time)+'.nii.gz'), os.path.join(new_folder,str(new_time)+'.nii.gz'))
#     shutil.copy2(os.path.join(folder,'img-nii-resampled-1.5mm',str(original_time)+'.nii.gz'), os.path.join(new_folder_resample,str(new_time)+'.nii.gz'))

#### Step 8: use patient_list_selected.xlsx as patient list and use voxelmorph to get MVF

#### Step 9: get sampled time frames
we should know that although ED is default to 0 but the actually max time frame may not be 0

In [4]:
spreadsheet = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_selected.xlsx'))
results = []
for i in range(0,spreadsheet.shape[0]):
    patient_class = spreadsheet.iloc[i]['patient_class']
    patient_id = spreadsheet.iloc[i]['patient_id']
   
    print('patient_class:', patient_class, ' patient_id:', patient_id)
    # load segmentation
    seg_folder = os.path.join('/mnt/camca_NAS/4DCT/mgh_data/predicted_seg/',patient_class,patient_id)
    seg_files = ff.sort_timeframe(ff.find_all_target_files(['*.nii.gz'],seg_folder),2,'_')
    total_tf_num = len(seg_files)
    
    # check shape consistency
    seg_volumes = []
    for ii in range(0, seg_files.shape[0]):
        img = nb.load(seg_files[ii]).get_fdata(); img = np.round(img).astype(np.uint8)
        seg_volumes.append(img)
    seg_volumes = np.transpose(np.asarray(seg_volumes),(1,2,3,0))

    # get a list of LV volumes (pixel value = 1) for all timeframes
    LV_volume_list = [np.sum(seg_volumes[:,:,:,i]==1) for i in range(0,seg_volumes.shape[-1])]
    LV_volume_list = np.asarray(LV_volume_list)

    # get ES time frame
    es_index = np.where(np.array(LV_volume_list) == np.min(LV_volume_list))[0][0]
    ejection_fraction = (LV_volume_list[0] - LV_volume_list[es_index])/LV_volume_list[0]
    last_tf_percent = (LV_volume_list[0] - LV_volume_list[-1])/LV_volume_list[0]

    # turn the time frame list into [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    # first find out the normalized index of es
    es_index_normalized = round(es_index/(total_tf_num),1)

    # sample the temporal series
    normalized_time_frame_list = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]; normalized_time_frame_list_copy = normalized_time_frame_list.copy()
    sampled_time_frame_list = []
    for t in range(0,len(normalized_time_frame_list)):
        # find the closest time frame to the normalized time frame
        tf_index = int(round(normalized_time_frame_list[t]*total_tf_num))
        if tf_index >= total_tf_num:
            tf_index = total_tf_num - 1
        sampled_time_frame_list.append(tf_index)

    # also calculate if pick [0.1,0.3,0.5,0.7,0.9], the ejection fraction is?
    # calculate the ejection fraction directly using semgnetaiton at each time frame
    picked_tf_normalized = [0.1,0.3,0.5,0.7,0.9]
    if len(sampled_time_frame_list)< 10:
        ejection_fraction_picked_5tf = ''
    else:
        picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]
        picked_LV_volume_list = [LV_volume_list[picked_tf[iii]] for iii in range(0,len(picked_tf))]
        ejection_fraction_picked_5tf_from_seg = (LV_volume_list[0] - np.min(picked_LV_volume_list))/LV_volume_list[0]
    # print('volume list: ', picked_LV_volume_list)
    # print('ejection_fraction_picked_5tf_from_seg: ', ejection_fraction_picked_5tf_from_seg)


    # also calculate if pick [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], the ejection fraction is?
    # calculate the ejection fraction directly using semgnetaiton at each time frame
    picked_tf_normalized = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    if len(sampled_time_frame_list)< 10:
        ejection_fraction_picked_10tf = ''
    else:
        picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]
        picked_LV_volume_list = [LV_volume_list[picked_tf[iii]] for iii in range(0,len(picked_tf))]
        ejection_fraction_picked_10tf_from_seg = (LV_volume_list[0] - np.min(picked_LV_volume_list))/LV_volume_list[0]

    # also calculate the ejection fraction using segmentation at ED 0 and deformation field at each time frame
    if len(sampled_time_frame_list)< 10:
        ejection_fraction_picked_5tf_from_mvf = ''
        ejection_fraction_picked_10tf_from_mvf = ''
    else:
        picked_tf = [sampled_time_frame_list[normalized_time_frame_list.index(picked_tf_normalized[iii])] for iii in range(0,len(picked_tf_normalized))]
        seg_template = nb.load(os.path.join(seg_folder, 'pred_s_0.nii.gz')).get_fdata()
        seg_template = np.round(seg_template).astype(np.int16)  # make sure the seg_template is in int16 format
        seg_template[seg_template != 1] = 0
        seg_template = Data_processing.crop_or_pad(seg_template, [160,160,96], value = 0)

        mvf_folder = os.path.join('/mnt/camca_NAS/4DCT/mvf_warp0_onecase/',patient_class, patient_id, 'voxel_final')
        volume_list = []
        volume_list = []
        seg_template_torch = torch.from_numpy(seg_template).unsqueeze(0).unsqueeze(0).float().cuda()
        for tf_n in range(0,len(picked_tf)):
            mvf_file = os.path.join(mvf_folder, str(picked_tf[tf_n]) + '.nii.gz')
            mvf_data = nb.load(mvf_file).get_fdata()
            mvf_data_torch = torch.from_numpy(np.transpose(mvf_data, (3, 0, 1, 2))).unsqueeze(0).float().cuda()
            warped_seg = warp_segmentation_from_mvf(seg_template_torch, mvf_data_torch)
            warped_seg = warped_seg.squeeze(0).squeeze(0).cpu().numpy()
            # warped_seg = Data_processing.apply_deformation_field_numpy(np.copy(seg_template), mvf_data)
            volume_list.append(np.sum(warped_seg))
        # print('volume_list: ', volume_list)
        volume_list = np.asarray(volume_list)
        ejection_fraction_picked_10tf_from_mvf = (LV_volume_list[0] - np.min(volume_list))/LV_volume_list[0]
        # print('ejection_fraction_picked_10tf_from_mvf: ', ejection_fraction_picked_10tf_from_mvf)
        volume_list_5tf = np.asarray([volume_list[1], volume_list[3], volume_list[5], volume_list[7], volume_list[9]]).reshape(-1)
        ejection_fraction_picked_5tf_from_mvf = (LV_volume_list[0] - np.min(volume_list_5tf))/LV_volume_list[0]
        # print('ejection_fraction_picked_5tf_from_mvf: ', ejection_fraction_picked_5tf_from_mvf)


    # how to assert that there is no duplicate in sampled_time_frame_list
    assert len(sampled_time_frame_list) == len(set(sampled_time_frame_list))
    # turn LV_volume_list back to a list
    LV_volume_list = LV_volume_list.tolist()

    # # append the results to the results list
    results.append([patient_class, patient_id, total_tf_num, es_index, es_index_normalized, sampled_time_frame_list, normalized_time_frame_list_copy, LV_volume_list, ejection_fraction, last_tf_percent, ejection_fraction_picked_5tf_from_seg, ejection_fraction_picked_5tf_from_mvf, ejection_fraction_picked_10tf_from_seg, ejection_fraction_picked_10tf_from_mvf])

    df = pd.DataFrame(results, columns = ['patient_class', 'patient_id', 'total_tf_num', 'es_index', 'es_index_normalized', 'sampled_time_frame_list', 'normalized_time_frame_list_copy', 'LV_volume_list', 'EF_original', 'last_tf_percent', 'EF_sampled_in_5tf_by_seg', 'EF_sampled_in_5tf_by_mvf', 'EF_sampled_in_10tf_by_seg', 'EF_sampled_in_10tf_by_mvf'])
    df.to_excel(os.path.join('/mnt/camca_NAS/4DCT/','Patient_lists/mgh/patient_list_final_selection_timeframes.xlsx'), index = False) 

patient_class: Pull_1  patient_id: case_1


#### step 10: check mvf max and min

In [13]:
df = pd.read_excel(os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_selected.xlsx'))
results = []
for ii in range(150,df.shape[0]):
    patient_class = df.iloc[ii]['patient_class']
    patient_id = df.iloc[ii]['patient_id']
    print(ii, patient_class, patient_id)

    mvf_folder = os.path.join('/mnt/camca_NAS/4DCT/mvf_warp0_onecase/', patient_class, patient_id, 'voxel_final')
    files = ff.find_all_target_files(['*.nii.gz'],mvf_folder)
    final_files = np.copy(files)
    for file in files:
        if 'moved' in file or 'original' in file:
            # remove it from the numpy array
            final_files = np.delete(final_files, np.where(final_files == file))
    files = ff.sort_timeframe(final_files,2)
    
    mvf = np.zeros((160,160,96,3,len(files)))
    for t in range(0,len(files)):
        img = nb.load(files[t]).get_fdata()
        mvf[:,:,:,:,t] = img
    results.append([patient_class, patient_id, np.max(mvf), np.min(mvf)])
    df1 = pd.DataFrame(results, columns=['patient_class', 'patient_id', 'max', 'min'])
    df1.to_excel('/mnt/camca_NAS/4DCT/Patient_lists/mgh/check_mvf_max_min.xlsx', index=False)
    

150 Pull_4 case_185
151 Pull_4 case_188
152 Pull_4 case_190
153 Pull_4 case_191
154 Pull_4 case_192
155 Pull_4 case_193
156 Pull_4 case_195
157 Pull_4 case_196
158 Pull_4 case_197
159 Pull_4 case_198
160 Pull_4 case_199
161 Pull_4 case_200
162 Pull_4 case_201
163 Pull_4 case_202
164 Pull_4 case_203
165 Pull_4 case_204
166 Pull_4 case_205
167 Pull_4 case_206
168 Pull_4 case_207
169 Pull_4 case_208
170 Pull_4 case_209
171 Pull_4 case_210
172 Pull_4 case_211
173 Pull_4 case_213
174 Pull_4 case_215
175 Pull_4 case_216
176 Pull_4 case_217
177 Pull_4 case_218
178 Pull_4 case_219
179 Pull_4 case_220
180 Pull_4 case_221
181 Pull_4 case_222
182 Pull_4 case_223
183 Pull_4 case_224
184 Pull_4 case_226
185 Pull_4 case_227
186 Pull_4 case_230
187 Pull_4 case_231
188 Pull_4 case_232
189 Pull_4 case_233
190 Pull_4 case_234
191 Pull_4 case_236
192 Pull_4 case_237
193 Pull_4 case_238
194 Pull_4 case_239
195 Pull_4 case_240
196 Pull_4 case_241
197 Pull_4 case_243
198 Pull_4 case_244
199 Pull_4 case_245
